In [21]:
import os
import pickle
import models

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import seaborn as sns
from numpyro.diagnostics import hpdi

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site

from hbmep.config import Config
from hbmep.model.utils import Site as site
from scipy import stats

from models import NonHierarchicalBayesianModel

USER = os.environ["USER"]
 
MAX_THRESHOLD_CONST = 3.042 #CONSTANT WILL BE mean of rats for stim max / threshold for EVERY possible combination... ma.mean(ma.median((stim_max/ma.mean(a, axis=0)),axis = (1,2,3)),axis =0)
NUM_THRESHOLD_POINTS = 500

In [3]:
src = f"/home/andres/repos/rat-mapping-paper/reports/J_SHAP/non_hierarchical_bayesian_model/inference.pkl"
with open(src, "rb") as f:
    (
        df,
        encoder_dict,
        model,
        posterior_samples,
    ) = pickle.load(f)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [26]:
a = posterior_samples[site.a]
b = posterior_samples[site.b]
L = posterior_samples[site.L]
ell = posterior_samples[site.ell]
H = posterior_samples[site.H]

x = np.linspace(0, 500, 1000)

named_params = [site.a, site.b, site.L, site.ell, site.H]
params = [posterior_samples[param][ :400,...] for param in named_params]

for named_param, param in zip(named_params, params):
    
    print(named_param, param.shape)

a (400, 8, 16, 4, 6)
b (400, 8, 16, 4, 6)
L (400, 8, 16, 4, 6)
ℓ (400, 8, 16, 4, 6)
H (400, 8, 16, 4, 6)


In [9]:
subjects = sorted(df['participant'].unique())
# subjects = encoder_dict['participant'].inverse_transform(subjects)
positions = sorted(df['compound_position'].unique())
# positions = encoder_dict['compound_position'].inverse_transform(positions)
charges = sorted(df['compound_charge_params'].unique())
# charges = encoder_dict['compound_charge_params'].inverse_transform(charges)
muscles = model.response

In [8]:
compound_position = encoder_dict[model.features[1]].inverse_transform(sorted(df['compound_position'].unique()))
mono = [c for c in compound_position if c.split("-")[0] == "" or c.split("-")[1] == ""]
mono = np.array(mono)
mono_inds = encoder_dict[model.features[1]].transform(mono)

# Get Constant

In [11]:
stim_max = []

for s in subjects:
    for p in mono_inds:
        for c in charges:
            for m in muscles: 
                temp = df[(df['participant'] == s) 
                        & (df['compound_position'] == p) 
                        & (df['compound_charge_params'] == c)
                        ]
                temp = temp.loc[:,[m, 'pulse_amplitude']]
                temp = temp.pulse_amplitude.max()
                stim_max.append(temp)
stim_max = ma.array(stim_max)
stim_max = stim_max.reshape(len(subjects), len(mono_inds), len(charges), len(muscles))

In [12]:
stim_max.shape

(8, 6, 4, 6)

In [13]:
a = posterior_samples[site.a]

In [14]:
a.shape

(4000, 8, 16, 4, 6)

In [15]:
a = a[:,:,mono_inds,:,:]

In [19]:
(stim_max/ma.mean(a, axis=0)).shape

(8, 6, 4, 6)

In [20]:
ma.mean(ma.median((stim_max/ma.mean(a, axis=0)),axis = (1,2,3)),axis = 0)

3.042548825398028

# Selectivity Calc

In [30]:
norm_y = []
norm_x = []

for subject_ind, subject in enumerate(subjects):
    for m in mono_inds:
        for c in charges:
            # Fix the subject index and size index
            curr_params = [
                params[:, subject_ind, mono_inds, :,:, None] for params in params
            ]

            # Take nanmean across posterior samples
            temp = np.nanmean(curr_params[0], axis=0)
            # Take nanmedian across compound positions and muscles
            median_threshold = np.nanmedian(temp)

            x_temp = np.linspace(0.,  median_threshold * MAX_THRESHOLD_CONST, NUM_THRESHOLD_POINTS)
            x_temp = x_temp[ None, None, None, None,:]

            temp_thresh = F.rectified_logistic(
                x_temp, #x_temp
                *curr_params
            )
            # Remove offset
            temp_thresh = temp_thresh - curr_params[2]
            norm_y.append(temp_thresh)
            norm_x.append(x_temp)

norm_x = np.array(norm_x)
norm_x = norm_x.reshape(len(subjects), len(mono_inds), len(charges),*norm_x.shape[1:])
norm_y = np.array(norm_y)
norm_y = norm_y.reshape(len(subjects),len(mono_inds), len(charges), *norm_y.shape[1:])
norm_y = ma.masked_invalid(norm_y)

/tmp/ipykernel_2784017/3389482008.py:13: RuntimeWarning: Mean of empty slice
  temp = np.nanmean(curr_params[0], axis=0)
/tmp/ipykernel_2784017/3389482008.py:13: RuntimeWarning: Mean of empty slice
  temp = np.nanmean(curr_params[0], axis=0)


In [33]:
norm_x.shape

(8, 6, 4, 1, 1, 1, 1, 500)

In [24]:
a.shape

(4000, 8, 6, 4, 6)